In [1]:
import numpy as np

In [2]:
# read in the text, transforming everything to lower case
text_org = open('Beiras.txt', encoding="utf-8").read().lower()
print('our original text has ' + str(len(text_org)) + ' characters')

our original text has 1213561 characters


In [3]:
import re
text_without_source="";
regexp=re.compile(r'http')
for line in text_org.splitlines():
    if not regexp.search(line):
        text_without_source= text_without_source + line

In [4]:
chars=sorted(list(set(text_without_source)))
print(chars);

[' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¨', 'ª', '«', '\xad', '´', 'º', '»', '¿', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '•', '…']


In [5]:

text_clean = re.sub('[ºªàâäçèêïìôöü&%@•…«»”“*/!"(),.:;_¿¡¿‘’´\[\]\']',' ',text_without_source)
text_clean = text_clean.replace("  "," ")
chars=sorted(list(set(text_clean )))
print(chars);
print ("this corpus has " +  str(len(text_clean)) + " total number of characters")
print ("this corpus has " +  str(len(chars)) + " unique characters")


[' ', '$', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¨', '\xad', 'á', 'ã', 'é', 'ë', 'í', 'î', 'ñ', 'ò', 'ó', 'õ', 'ú', '–', '—']
this corpus has 1174380 total number of characters
this corpus has 55 unique characters


In [6]:
def window_transform_text(text,window_size,step_size):
    # containers for input/output pairs
    inputs = []
    outputs = []
    #Number of windows to create
    n_windows=int((len(text) - window_size)/ step_size)
    for j in range(n_windows) :
        # k .- Start index
        k= j * step_size
        inputs.append(text[k:(k+window_size)])
        outputs.append(text[k+window_size])

    return inputs,outputs

In [7]:
# run your text window-ing function 
window_size = 100
step_size = 1

In [8]:
# this dictionary is a function mapping each unique character to a unique integer
chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer

# this dictionary is a function mapping each unique integer back to a unique character
indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character

In [9]:
# transform character-based input/output into equivalent numerical versions
def encode_io_pairs(text,window_size,step_size):
    # number of unique chars
    chars = sorted(list(set(text)))
    num_chars = len(chars)
    
    # cut up text into character input/output pairs
    inputs, outputs = window_transform_text(text,window_size,step_size)
    
    # create empty vessels for one-hot encoded input/output
    X = np.zeros((len(inputs), window_size, num_chars), dtype=np.bool)
    y = np.zeros((len(inputs), num_chars), dtype=np.bool)
    
    # loop over inputs/outputs and tranform and store in X/y
    for i, sentence in enumerate(inputs):
        for t, char in enumerate(sentence):
            X[i, t, chars_to_indices[char]] = 1
        y[i, chars_to_indices[outputs[i]]] = 1
        
    return X,y

In [10]:
# use your function
X,y = encode_io_pairs(text_clean,window_size,step_size)

In [11]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15587238462670927579, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 3879469056
 locality {
   bus_id: 1
 }
 incarnation: 17891074825725798988
 physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"]

In [12]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [13]:
Xsmall = X[:10000,:,:]
ysmall = y[:10000,:]

In [14]:
from keras.optimizers import Adam
from keras.layers import Input
from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers.core import Dense
from keras.layers import Concatenate,concatenate,Reshape

def create_gan_simple_model(chars,shp):
    num_chars = len(chars)
    opt = Adam(lr=1e-4)
    dopt = Adam(lr=1e-3)
    
    g_input = Input(shape=shp)
    H=GRU(200)(g_input)
    g_V=Dense(num_chars,activation='softmax')(H)
    generator = Model(g_input,g_V)
    generator.compile(loss='binary_crossentropy', optimizer=opt)
    generator.summary()
    
    shp_discriminator=(shp[0]+1,shp[1])
    print(shp_discriminator)
    d_input = Input(shape=shp_discriminator)
    H=GRU(200)(d_input)
    d_V=Dense(2,activation='softmax')(H)
    discriminator = Model(d_input,d_V)
    discriminator.compile(loss='categorical_crossentropy', optimizer=dopt)
    discriminator.summary()
        
    make_trainable(discriminator, False)
    
    gan_input = Input(shape=shp)
    H = generator(gan_input)
    print(gan_input.shape,H.shape)
    print(H.shape[1],num_chars)
    H = Reshape((1,num_chars))(H)
    print(gan_input.shape,H.shape)
    H=concatenate([gan_input,H],axis=1)
    print(H.shape)
    gan_V = discriminator(H)
    GAN = Model(gan_input, gan_V)
    GAN.compile(loss='categorical_crossentropy', optimizer=opt)
    GAN.summary()
    
    return generator,discriminator,GAN

Using TensorFlow backend.


In [15]:
generetor,discriminator,GAN=create_gan_simple_model(chars,X.shape[1:])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 55)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 200)               153600    
_________________________________________________________________
dense_1 (Dense)              (None, 55)                11055     
Total params: 164,655
Trainable params: 164,655
Non-trainable params: 0
_________________________________________________________________
(101, 55)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 101, 55)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 200)               153600    
____________________________________________________________